In [1]:
# saves you having to use print as all exposed variables are printed in the cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [47]:
import re
import math
import pandas as pd
import numpy as np
import torch
import pyarrow as pa
import pyarrow.compute as pc
import ast
from nn_rag import Knowledge
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer, util
from collections import Counter

In [10]:
kn = Knowledge.from_memory()

In [43]:
def score(t1, t2):
    embedding_model = SentenceTransformer(model_name_or_path='all-mpnet-base-v2')
    v1 = embedding_model.encode(t1)
    v2 = embedding_model.encode(t2)
    return util.dot_score(v1, v2)[0,0].tolist()

In [44]:
text1 = "This is a foo bar sentence ."
text2 = "This sentence is similar to a foo bar sentence ."

score(text1, text2)

0.9069209098815918

In [46]:
text1 = 'You took too long.'
text2 = 'You were very slow'

score(text1, text2)

0.4604153633117676